In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import pandas as pd

In [3]:
dataset = pd.read_json("../Datasets/Sarcasm_Headlines_Dataset.json", lines=True)
dataset.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [4]:
column_name_to_remove = 'article_link'
dataset = dataset.drop(columns=[column_name_to_remove])

dataset.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [9]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences


X_train, X_test, y_train, y_test = train_test_split(dataset['headline'], dataset['is_sarcastic'], test_size=0.2, random_state=42)

# Tokenize and vectorize the training text data using Tokenizer and pad_sequences
max_length = 100
tokenizer = Tokenizer()   #lower=False
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=max_length)

# Tokenize and vectorize the testing text data using the same Tokenizer
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=max_length)

In [10]:
from keras.layers import CuDNNLSTM
from keras.models import Sequential
from keras.layers import Embedding, Dense
from keras.optimizers import Adam

embedding_dim = 100

# Define the vocabulary size based on the actual number of unique words in the training data
vocab_size = len(tokenizer.word_index) + 1

max_length = 100
optimizer = Adam(learning_rate=0.00001)
m1 = Sequential()
m1.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
m1.add(CuDNNLSTM(units=150))
m1.add(Dense(units=64))
m1.add(Dense(units=64))
m1.add(Dense(units=1, activation='sigmoid'))

m1.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
m1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          2653600   
                                                                 
 cu_dnnlstm_1 (CuDNNLSTM)    (None, 150)               151200    
                                                                 
 dense_3 (Dense)             (None, 64)                9664      
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,818,689
Trainable params: 2,818,689
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Train the model
m1.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = m1.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy * 100:.2f}%')

Epoch 1/20
334/334 [==============================] - 6s 15ms/step - loss: 0.6879 - accuracy: 0.5592 - val_loss: 0.6836 - val_accuracy: 0.5608
Epoch 2/20
334/334 [==============================] - 5s 14ms/step - loss: 0.6815 - accuracy: 0.5611 - val_loss: 0.6789 - val_accuracy: 0.5608
Epoch 3/20
334/334 [==============================] - 5s 14ms/step - loss: 0.6727 - accuracy: 0.5611 - val_loss: 0.6642 - val_accuracy: 0.5608
Epoch 4/20
334/334 [==============================] - 5s 14ms/step - loss: 0.6334 - accuracy: 0.6150 - val_loss: 0.5942 - val_accuracy: 0.7192
Epoch 5/20
334/334 [==============================] - 5s 14ms/step - loss: 0.5340 - accuracy: 0.7818 - val_loss: 0.5012 - val_accuracy: 0.7954
Epoch 6/20
334/334 [==============================] - 5s 14ms/step - loss: 0.4419 - accuracy: 0.8279 - val_loss: 0.4413 - val_accuracy: 0.8122
Epoch 7/20
334/334 [==============================] - 5s 14ms/step - loss: 0.3820 - accuracy: 0.8506 - val_loss: 0.4101 - val_accuracy: 0.8184

In [13]:
from sklearn.metrics import precision_score, recall_score

# Predict on validation data
y_val_pred_prob_m1 = m1.predict(X_test)
y_val_pred_m1 = (y_val_pred_prob_m1 > 0.5).astype(int)  # Threshold for binary classification

# Assuming y_test is in binary format (0 or 1)
y_val_true_m1 = y_test

# Calculate precision and recall for binary classification
precision_m1 = precision_score(y_val_true_m1, y_val_pred_m1)
recall_m1 = recall_score(y_val_true_m1, y_val_pred_m1)

# print the results
print(f'Precision: {precision_m1:.4f}')
print(f'Recall: {recall_m1:.4f}')

167/167 [==============================] - 1s 4ms/step
Precision: 0.8333
Recall: 0.8457
